In [1]:
import pandas as pd
import spacy
import os
import spacy
import polars as pl
import mlflow
import mlflow.spacy
from pathlib import Path
import json
from spacy.util import minibatch, compounding
from spacy.cli.train import train
from dotenv import load_dotenv
import sys
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../..'))
from app.data_handling.preprocessing import process_ingredient, transform_ingredients_to_tokens

# Create Dataframe and save as parquet

In [2]:
load_dotenv()

True

In [3]:
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URL"))

In [4]:
model_uri = "models:/recipe_NER@prod"
ner = mlflow.spacy.load_model(model_uri=model_uri)

/home/naabzer/develop/recipe_tm/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [302]:
nlg_ds = pd.read_csv('../../data/recipenlg/RecipeNLG_dataset.csv', converters={'ingredients': pd.eval}, nrows=1000)

In [324]:
nlg_ds = pl.read_csv('../../data/recipenlg/RecipeNLG_dataset.csv',  n_rows=1000)
nlg_ds = nlg_ds.with_columns(pl.col("ingredients").str.json_decode())
nlg_ds = nlg_ds.with_columns(pl.col("directions").str.json_decode())

In [325]:
nlg_ds

,title,ingredients,directions,link,source,NER
i64,str,list[str],list[str],str,str,str
0,"""No-Bake Nut Cookies""","[""1 c. firmly packed brown sugar"", ""1/2 c. evaporated milk"", … ""3 1/2 c. bite size shredded rice biscuits""]","[""In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine."", ""Stir over medium heat until mixture bubbles all over top."", … ""Let stand until firm, about 30 minutes.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""brown sugar"", ""milk"", ""vanil…"
1,"""Jewell Ball'S Chicken""","[""1 small jar chipped beef, cut up"", ""4 boned chicken breasts"", … ""1 carton sour cream""]","[""Place chipped beef on bottom of baking dish."", ""Place chicken on top of beef."", ""Mix soup and cream together; pour over chicken. Bake, uncovered, at 275° for 3 hours.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""beef"", ""chicken breasts"", ""c…"
2,"""Creamy Corn""","[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg. cream cheese, cubed"", … ""1/4 tsp. pepper""]","[""In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""frozen corn"", ""cream cheese""…"
3,"""Chicken Funny""","[""1 large whole chicken"", ""2 (10 1/2 oz.) cans chicken gravy"", … ""4 oz. shredded cheese""]","[""Boil and debone chicken."", ""Put bite size pieces in average size square casserole dish."", … ""Sprinkle shredded cheese on top and bake at 350° for approximately 20 minutes or until golden and bubbly.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""chicken"", ""chicken gravy"", ""…"
4,"""Reeses Cups(Candy) ""","[""1 c. peanut butter"", ""3/4 c. graham cracker crumbs"", … ""1 large pkg. chocolate chips""]","[""Combine first four ingredients and press in 13 x 9-inch ungreased pan."", ""Melt chocolate chips and spread over mixture. Refrigerate for about 20 minutes and cut into pieces before chocolate gets hard."", ""Keep in refrigerator.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""peanut butter"", ""graham crac…"
…,…,…,…,…,…,…
995,"""Heath Bar Pie""","[""3 Heath bars, chopped fine"", ""1 medium container whipping topping"", … ""chocolate curls or chopped nuts""]","[""Mix chopped Heath bars with whipped topping and pour into crust."", ""Top with chocolate curls or chopped nuts."", ""Refrigerate.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""graham cracker pie crust"", ""…"
996,"""Victorian Baked French Toast""","[""1 c. brown sugar"", ""1/3 c. butter"", … ""1 tsp. cinnamon""]","[""Cook brown sugar, butter and corn syrup in small saucepan until butter melts."", ""Pour into 9 x 13-inch pan."", … ""Bake at 350° for 30 minutes or until center is done and lightly browned.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""brown sugar"", ""butter"", ""lig…"
997,"""Quick Swedish Meatballs""","[""1 lb. ground beef"", ""1 c. soft bread crumbs"", … ""1 pkg. stroganoff sauce mix""]","[""Combine meat, bread crumbs, cheese, soup mix, seasonings and 1/4 cup milk; mix thoroughly (with hands)."", ""Form into small balls. Brown lightly; remove to a baking dish."", … ""Bake in a 350° oven for 30 to 45 minutes.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""ground beef"", ""bread crumbs""…"


In [326]:
nlg_ds['ingredients'][0]


""
str
"""1 c. firmly packed brown sugar"""
"""1/2 c. evaporated milk"""
"""1/2 tsp. vanilla"""
"""1/2 c. broken nuts (pecans)"""
"""2 Tbsp. butter or margarine"""
"""3 1/2 c. bite size shredded ri…"


In [327]:
def transform_ds_to_BOW(ds: pl.dataframe.frame.DataFrame, create_vocab=False):  # Assume recipeNLG dataset, where each ingredient line is a list item
    tokens = []
    preps = []
    optionals = []
    for i, ingredients in enumerate(ds['ingredients']):
        datapoint_obj = transform_ingredients_to_tokens(ingredients, ner, create_vocab)
        tokens.append(datapoint_obj['foods'])
        preps.append(json.dumps(datapoint_obj['preps']))
        optionals.append(datapoint_obj['optionals'])
    new_ds = ds.with_columns(
        pl.Series(name='BOW_tokens', values=tokens),
        pl.Series(name='preps', values=preps),
        pl.Series(name='optionals', values=optionals)
    )
    return new_ds

In [328]:
nlg_ds.head()

,title,ingredients,directions,link,source,NER
i64,str,list[str],list[str],str,str,str
0,"""No-Bake Nut Cookies""","[""1 c. firmly packed brown sugar"", ""1/2 c. evaporated milk"", … ""3 1/2 c. bite size shredded rice biscuits""]","[""In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine."", ""Stir over medium heat until mixture bubbles all over top."", … ""Let stand until firm, about 30 minutes.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""brown sugar"", ""milk"", ""vanil…"
1,"""Jewell Ball'S Chicken""","[""1 small jar chipped beef, cut up"", ""4 boned chicken breasts"", … ""1 carton sour cream""]","[""Place chipped beef on bottom of baking dish."", ""Place chicken on top of beef."", ""Mix soup and cream together; pour over chicken. Bake, uncovered, at 275° for 3 hours.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""beef"", ""chicken breasts"", ""c…"
2,"""Creamy Corn""","[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg. cream cheese, cubed"", … ""1/4 tsp. pepper""]","[""In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""frozen corn"", ""cream cheese""…"
3,"""Chicken Funny""","[""1 large whole chicken"", ""2 (10 1/2 oz.) cans chicken gravy"", … ""4 oz. shredded cheese""]","[""Boil and debone chicken."", ""Put bite size pieces in average size square casserole dish."", … ""Sprinkle shredded cheese on top and bake at 350° for approximately 20 minutes or until golden and bubbly.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""chicken"", ""chicken gravy"", ""…"
4,"""Reeses Cups(Candy) ""","[""1 c. peanut butter"", ""3/4 c. graham cracker crumbs"", … ""1 large pkg. chocolate chips""]","[""Combine first four ingredients and press in 13 x 9-inch ungreased pan."", ""Melt chocolate chips and spread over mixture. Refrigerate for about 20 minutes and cut into pieces before chocolate gets hard."", ""Keep in refrigerator.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""peanut butter"", ""graham crac…"


In [329]:
nlg_ds = transform_ds_to_BOW(nlg_ds, create_vocab=True)

In [330]:
nlg_ds.write_parquet('test.parquet')

# Test some queries using DuckDB and parquet

In [331]:
import duckdb

In [332]:
duckdb.sql("SELECT * FROM 'test.parquet' AS tbl WHERE list_contains(tbl.BOW_tokens, 'egg')").pl()

C0,title,ingredients,directions,link,source,NER,BOW_tokens,preps,optionals
i64,str,list[str],list[str],str,str,str,list[str],str,list[bool]
6,"""Rhubarb Coffee Cake""","[""1 1/2 c. sugar"", ""1/2 c. butter"", … ""1 tsp. vanilla""]","[""Cream sugar and butter."", ""Add egg and beat well."", … ""Pour into greased 9 x 13-inch pan and add Topping.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""sugar"", ""butter"", ""egg"", ""bu…","[""sugar"", ""butter"", … ""rhubarb""]","""{""rhubarb"": [""finely"", ""cut""]}""","[false, false, … false]"
7,"""Scalloped Corn""","[""1 can cream-style corn"", ""1 can whole kernel corn"", … ""pepper to taste""]","[""Mix together both cans of corn, crackers, egg, 2 teaspoons of melted butter and pepper and place in a buttered baking dish."", ""Dot with remaining 4 teaspoons of butter."", ""Bake at 350° for 1 hour.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""cream-style corn"", ""whole ke…","[""cream_-_style"", ""corn"", … ""pepper""]","""{""saltine_cracker"": [""crush""],…","[false, false, … false]"
20,"""Grandma Hanrath'S Banana Bread…","[""1 c. sugar"", ""1/2 c. shortening"", … ""2 over-ripe bananas (chopped)""]","[""Cream sugar and shortening."", ""Add eggs, salt and soda, then bananas and flour."", … ""Bake in loaf pan at 350° for 1 hour.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""sugar"", ""shortening"", ""eggs""…","[""sugar"", ""egg"", … ""banana""]","""{""banana"": [""chop""]}""","[false, false, … false]"
21,"""Chocolate Frango Mints""","[""1 pkg. devil's food cake mix"", ""1 pkg. chocolate fudge pudding mix (instant)"", … ""6 oz. chopped Frango mints""]","[""Mix ingredients together for 5 minutes."", ""Scrape bowl often. Last fold in chocolate chip mints."", ""Bake at 350° for 35 to 40 minutes or until done (cake mix directions).""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""cake mix"", ""chocolate fudge …","[""cake_mix"", ""sour_cream"", ""egg""]","""{}""","[false, false, false]"
24,"""Prize-Winning Meat Loaf""","[""1 1/2 lb. ground beef"", ""1 c. tomato juice"", … ""1 1/2 tsp. salt""]","[""Mix well."", ""Press firmly into an 8 1/2 x 4 1/2 x 2 1/2-inch loaf pan."", … ""Makes 8 servings.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""ground beef"", ""tomato juice""…","[""beef"", ""tomato_juice"", … ""salt""]","""{""beef"": [""ground""], ""egg"": [""…","[false, false, … false]"
…,…,…,…,…,…,…,…,…,…
987,"""French Toast And Sauce""","[""4 eggs, beaten"", ""1/2 c. milk"", … ""8 slices day-old bread""]","[""In a wide shallow bowl, beat 1st four ingredients. For each slice, melt 1 tablespoon of butter in skillet. Dip bread in egg mixture; turn and coat the other side. Cook slices in butter, browning on both sides. Serve with Sauce as follows.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""eggs"", ""milk"", ""frozen lemon…","[""egg"", ""milk"", … ""bread""]","""{""egg"": [""beat""]}""","[false, false, … false]"
988,"""Spaghetti Pie""","[""6 oz. spaghetti"", ""2 Tbsp. olive oil"", … ""1/2 c. shredded Mozzarella (2 oz.)""]","[""Cook spaghetti (about 3 cups)."", ""Stir olive oil into hot spaghetti."", … ""Form spaghetti mixture into a ""crust"" in a 10-inch pie plate sprayed with Pam. Spread cottage cheese over bottom of spaghetti crust.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""spaghetti"", ""olive oil"", ""Pa…","[""olive_oil"", ""parmesan_cheese"", … ""mozzarella""]","""{""parmesan_cheese"": [""grate""],…","[false, false, … false]"
991,"""Favorite Chocolate Cake""","[""1 3/4 c. flour"", ""2 eggs"", … ""1 c. buttermilk""]","[""Sift all dry ingredients in a large mixing bowl."", ""Stir."", … ""Tube pan takes 40 minutes.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""flour"", ""eggs"", ""baking powd…","[""egg"", ""bake_powder"", … ""buttermilk""]","""{}""","[false, false, … false]"


In [333]:
duckdb.sql("COPY(SELECT preps::JSON AS preps, * EXCLUDE (preps) FROM 'test.parquet') TO 'test.parquet' (FORMAT PARQUET, OVERWRITE TRUE)")

In [334]:
duckdb.sql("SELECT json_extract(preps, ['butter', 'rhubarb']) FROM 'test.parquet'")

┌───────────────────────────────────────────────────────────┐
│ json_extract(preps, main.list_value('butter', 'rhubarb')) │
│                          json[]                           │
├───────────────────────────────────────────────────────────┤
│ [NULL, NULL]                                              │
│ [NULL, NULL]                                              │
│ ['["cube"]', NULL]                                        │
│ [NULL, NULL]                                              │
│ [NULL, NULL]                                              │
│ [NULL, NULL]                                              │
│ [NULL, '["finely","cut"]']                                │
│ [NULL, NULL]                                              │
│ [NULL, NULL]                                              │
│ [NULL, NULL]                                              │
│      ·                                                    │
│      ·                                                    │
│      ·

In [335]:
duckdb.sql("SELECT json_exists(preps, ['butter', 'rhubarb']) FROM 'test.parquet'")

┌──────────────────────────────────────────────────────────┐
│ json_exists(preps, main.list_value('butter', 'rhubarb')) │
│                        boolean[]                         │
├──────────────────────────────────────────────────────────┤
│ [false, false]                                           │
│ [false, false]                                           │
│ [true, false]                                            │
│ [false, false]                                           │
│ [false, false]                                           │
│ [false, false]                                           │
│ [false, true]                                            │
│ [false, false]                                           │
│ [false, false]                                           │
│ [false, false]                                           │
│       ·                                                  │
│       ·                                                  │
│       ·               

In [336]:
ingredients = ['butter']
preps = {'butter': ['cube'], 'cream_cheese': ['cube']}

In [337]:
list(preps.keys())

['butter', 'cream_cheese']

In [352]:
def construct_ingredient_query(pq_path: str, ingredients: list, preps: dict):
    prep_keys = list(preps.keys())
    base_sql = (f"SELECT *, json_extract(preps, {prep_keys}) AS prep_lists"
                f" FROM '{pq_path}'"
                " AS tbl"
                f" WHERE list_has_all(tbl.BOW_tokens, {ingredients})"
               )
    prep_filter = ""
    for i, (key, value) in enumerate(preps.items()):
        prep_filter += (f" AND (prep_lists->>{i}) IS NOT NULL"
                        f" AND list_has_all(from_json(prep_lists->>{i}, '[\"VARCHAR\"]'), {value})"
                       )
    sql = base_sql + prep_filter
    return sql

In [353]:
duckdb.sql(construct_ingredient_query('test.parquet', ingredients, preps)).pl()

preps,C0,title,ingredients,directions,link,source,NER,BOW_tokens,optionals,prep_lists
str,i64,str,list[str],list[str],str,str,str,list[str],list[bool],list[str]
"""{""almond"": [""sliver"", ""chop""]}""",141,"""Caramel Pie""","[""16 oz. Cool Whip"", ""2 c. coconut"", … ""1 jar caramel topping""]","[""Mix Cool Whip, softened cream cheese and Eagle Brand milk. Beat for 1 minute."", ""Pour in pie crusts."", … ""Freeze.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""coconut"", ""margarine"", ""sliv…","[""margarine"", ""almond"", … ""caramel_topping""]","[false, false, … false]","[""[""sliver"",""chop""]""]"
"""{""almond"": [""chop""]}""",182,"""Almond Roco""","[""1 lb. butter (not margarine)"", ""2 c. granulated sugar"", … ""1/4 c. chopped almonds or as desired (for topping)""]","[""Cook until hard ball stage in cold water."", ""Add 1/2 cup chopped almonds and cook until it reaches cracked candy stage, 300°, on candy thermometer."", ""Melt the 3 chocolate candy bars and pour on top and sprinkle chopped almonds.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""butter"", ""sugar"", ""water"", ""…","[""butter"", ""almond"", … ""almond""]","[false, false, … false]","[""[""chop""]""]"
"""{""green_pepper"": [""chop""], ""on…",479,"""Luau""","[""2 lb. pork sausage"", ""2 c. white Minute rice"", … ""2 dry packs Lipton noodle soup""]","[""Brown sausage."", ""Remove sausage and cook the pepper, onions and celery in as little grease as possible."", … ""Serves 10 to 12.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""pork sausage"", ""white Minute…","[""sausage"", ""minute_rice"", … ""pack_lipton_noodle_soup""]","[false, false, … false]","[""[""chop""]""]"


In [354]:
ingredients = ['almond']
preps = {}
sql = construct_ingredient_query('test.parquet', ingredients, preps)
results = duckdb.sql(sql).pl()
print("###############################")
print("####### WITHOUT PREPS #########")
print("###############################")
for title, ingr in results['title', 'ingredients'].iter_rows():
    print()
    print(f"---- {title} ----")
    print("\n".join(ingr))
    
ingredients = ['almond']
preps = {'almond': ['chop']}
sql = construct_ingredient_query('test.parquet', ingredients, preps)
results = duckdb.sql(sql).pl()
print("##############################")
print("######## WITH PREPS ##########")
print("##############################")
for title, ingr in results['title', 'ingredients'].iter_rows():
    print()
    print(f"---- {title} ----")
    print("\n".join(ingr))

###############################
####### WITHOUT PREPS #########
###############################

---- Honey Almond Squares ----
1 Duncan Hines white pudding cake mix
1 (8 oz.) pkg. cream cheese (at room temperature)
1/4 c. alfalfa honey
3 to 4 oz. sliced almonds

---- Caramel Pie ----
16 oz. Cool Whip
2 c. coconut
3/4 margarine, melted
1 c. slivered almonds, chopped fine
1 can Eagle Brand milk
3 graham cracker crusts
8 oz. cream cheese
1 jar caramel topping

---- Almond Roco ----
1 lb. butter (not margarine)
2 c. granulated sugar
1/2 c. water
1/2 c. chopped almonds
3 chocolate candy bars
1/4 c. chopped almonds or as desired (for topping)

---- Crabmeat Au Gratin ----
1 can cream of shrimp soup
1/2 can milk (use soup can)
1 lb. faux crabmeat
4 c. celery, chopped
1/2 c. green pepper, chopped
2 pimentos, chopped
2 Tbsp. onions, chopped
1/3 c. slivered almonds
4 hard-cooked eggs, chopped
1 c. shredded sharp Cheddar cheese
buttered bread crumbs

---- Fresh Broccoli Salad ----
2 bunches fres